# Webapp: set up data stream, visualize, and equitize

- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
- https://dash.plotly.com/live-updates
- https://pbpython.com/interactive-dashboards.html#id6
- https://mybinder.org/ + https://github.com/echow1/trading_music
- (maybe) https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/
- https://kapernikov.com/ipywidgets-with-matplotlib/

In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os, time, re
%matplotlib inline
from IPython.core.display import display, HTML
### change width of notebook display
display(HTML("<style>.container { width:70% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

FIG_WIDTH = 1200
FIG_HEIGHT = 800

PITCH_MIN = 20
PITCH_MAX = 120
VELOCITY_MIN = 0
VELOCITY_MAX = 120

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

### Setup: read in the music stream

In [5]:
""" Set up music stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

def nextChunk(csvStream, chunksize=5):
    return(csvStream.get_chunk(chunksize))

chunksQueue = []
maxChunksQueueSize = 120
def nextChunkWithOverlap(musStream, cq=chunksQueue, chunksize=5, maxNumChunks=maxChunksQueueSize):
    """
    Iterate over the music stream with rolling window.
    For smoother plotting, set chunksize <<< max number of chunks.
    """
    nextChunk = (musStream.get_chunk(chunksize))
    # make space
    if len(cq) >= maxNumChunks:
        cq.pop(0)
    cq.append(nextChunk)
    #should be sorted always because FIFO but maybe should check.
    res = (pd.concat(cq))
    return(res)

# def hasNext(csvStream):
    

INPUT_PATH = "data_processed/maestro/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x)
MUSIC_STREAM_SUBSTR = "maestro_full_music_stream"

music_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if MUSIC_STREAM_SUBSTR in file:
            music_files.append(os.path.join(root, file))

### pick first as the music stream
music_files = sorted(music_files) # play in order
print("Number of music streams found:")
print(len(music_files))
print(music_files[:10])

### should only 1 have file to stream
if (len(music_files) > 1):
    whichStream = int(input("Index (0 ... N-1) of stream to pick:"))
else:
    whichStream = 0
musicStream = csvStream(music_files[whichStream])

Number of music streams found:
1
['data_processed/maestro/maestro_full_music_stream.csv']


### Task 0: Stream music audio and display as a webapp

In [6]:
# Could just play synthesized MIDI along for now (and replace with real audio later)


### Task 1: Stream MIDI and display primitive music statistics (pitch, velocity etc.)

(do audio later after download)

In [8]:
# import jp_proxy_widget
# from scipy.io import wavfile

""" Demo: auto-updating time series plot, use with voila and watch update """

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

""" Set up plot.ly / dash plots to be updated automatically in real-time
"""
### Setup plots for time series
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("Music streaming statistics"),
    ### Graph 1: pitch
    dcc.Graph(id='pitch_graph'),
    ### Graph 2: velocity
    dcc.Graph(id='velocity_graph'),
    ### Graphs update automatically (separate from the main for-loops for data analysis)
    ### Make sure updates faster than the main for-loops (so don't miss any data updates)
    ### updates every [interval] milliseconds.
    dcc.Interval(id='interval-component', interval=0.5*1000, n_intervals=0)
# ])
],     style={'width': '80%', 'float': 'left', 'height': '4.5rem'})

### Plot #1: pitch info
@app.callback(Output('pitch_graph', 'figure'), Input("interval-component", "n_intervals"))
def update_pitch_figure(n=0):
    """
    Update plotly figure. (Like ggplot2: color based on group)
    currBar (global): variable with the current data.
    """
    fig = px.line(currBar, x="streaming_start_sec",
                  y=['pitch_min', 'pitch_mean', 'pitch_max'],
                  render_mode="webgl", template="plotly_dark",
        title="Pitch statistics by sampled bar, streaming",range_y=[PITCH_MIN, PITCH_MAX]).update_traces(mode='lines')
    return(fig)

### Plot #2: velocity info
@app.callback(Output('velocity_graph', 'figure'), Input("interval-component", "n_intervals"))
def update_velocity_figure(n=0):
    """
    Update plotly figure. (Like ggplot2: color based on group)
    currBar (global): variable with the current data.
    """
    fig = px.line(currBar, x="streaming_start_sec",
                  y=['velocity_min', 'velocity_mean', 'velocity_max'],
                  render_mode="webgl", template="plotly_dark",
        title="Velocity statistics by sampled bar, streaming",range_y=[VELOCITY_MIN, VELOCITY_MAX]).update_traces(mode='lines')
    return(fig)

### Run app locally (inline cuts off output)
app.run_server(mode='external')

while True:
    currBar = nextChunkWithOverlap(musicStream)
    if currBar is None:
        print(">> End of stream!")
        break
    ### Only start when queue is full (seed with initial data)
    if len(chunksQueue) < maxChunksQueueSize:
        continue
        
    """ Analysis with current bar here """
        
        
    
    ### Take a short break between analyses (so plotly can catch up)
    ### should be >> plot auto-update interval so that all plots
    ### update basically at the same time. 
    time.sleep(1)

Dash app running on http://127.0.0.1:8050/


KeyboardInterrupt: 

In [ ]:
raise Exception()

### Task 3: Extract music features from MIDI in real-time, construct financial equities, visualize

Sequential learning.
- Validate (try out) against the streamed music audio and series.
- These are the constructed underlyings for financial equities, upon which prediction/regression can work.

Make a local API so other scripts can GET/POST requests (bid/ask) for this.

In [9]:
""" TEMPO
    Strategy: linear Gaussian state space model / Kalman filter.
    Model tempo (latent variable zt) as a function of notes etc. (observed variables x1 ... xt)
    https://www.researchgate.net/publication/224711190_A_Modified_Kalman_Filtering_Approach_to_On-Line_Musical_Beat_Tracking
"""

test_data = pd.concat(chunksQueue)

In [10]:
test_data.head()

,streaming_start_sec,piece_num,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode,piece_changed
215,215,0,215,215.04479,215.491957,215.494270,215.95938,"215.04479,215.04479,215.15208,215.16042,215.24...",215.10625,215.563657,215.577085,216.01146,"215.10625,215.12708,215.24688,215.22083,215.30...",48.0,61.833333,61.0,76.0,"62,60,59,64,57,66,55,67,54,69,71,52,72,50,57,7...",66.0,72.055556,71.0,78.0,"71,73,66,71,73,78,73,78,70,75,70,71,76,72,68,7...",0.04063,0.071702,0.06719,0.11146,"0.06146,0.08229,0.09479,0.06042,0.05729,0.0677...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
216,216,0,216,216.06979,216.477964,216.438540,216.92708,"216.06979,216.07604,216.18125,216.19167,216.31...",216.13646,216.555528,216.508330,216.97396,"216.14062,216.13646,216.40625,216.29271,216.38...",45.0,66.923077,74.0,81.0,"78,57,47,79,81,45,74,78,79,47,81,50,74",59.0,69.769231,70.0,85.0,"76,63,85,71,73,73,70,65,69,74,66,63,59",0.03438,0.077565,0.07083,0.22500,"0.07083,0.06042,0.225,0.10104,0.07812,0.04271,...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
217,217,0,217,217.03854,217.475954,217.447395,217.89271,"217.03854,217.15833,217.16562,217.27396,217.38...",217.10417,217.564064,217.525525,217.97188,"217.10417,217.20833,217.20521,217.38646,217.43...",47.0,68.333333,75.0,81.0,"79,71,55,79,47,74,81,79,48,81,50,76",60.0,71.250000,71.5,81.0,"60,68,66,62,71,73,72,76,81,77,79,70",0.03542,0.088109,0.07240,0.24792,"0.06563,0.05,0.03958,0.1125,0.04688,0.05312,0....",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
218,218,0,218,218.01771,218.490865,218.487500,218.96771,"218.01771,218.12604,218.1375,218.25625,218.378...",218.09583,218.563062,218.564580,219.10521,"218.09583,218.16771,218.25417,218.33646,218.41...",45.0,68.615385,76.0,81.0,"78,52,79,81,48,76,79,45,72,79,48,76,79",66.0,74.307692,75.0,80.0,"66,75,76,75,69,75,73,72,77,74,76,80,78",0.03333,0.072195,0.06146,0.13750,"0.07812,0.04167,0.11667,0.08021,0.03333,0.0479...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
219,219,0,219,219.09271,219.526525,219.566670,219.96146,"219.09271,219.09792,219.21042,219.32396,219.33...",219.18333,219.615304,219.607290,220.04271,"219.33125,219.18333,219.32604,219.43333,219.37...",50.0,67.384615,72.0,81.0,"50,78,79,81,52,78,72,54,76,74,55,72,55",28.0,70.384615,74.0,79.0,"75,79,78,78,71,76,77,70,70,74,70,69,28",0.03542,0.088782,0.07812,0.23854,"0.23854,0.08542,0.11562,0.10938,0.04167,0.0781...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0


In [ ]:
""" HARMONY
"""

#

In [ ]:
""" RHYTHM
"""

#